In [1]:
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import figure
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [2]:
# Settings:
pd.set_option('display.width', 190)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('max_colwidth', 200)
pd.options.display.float_format = '{:.4f}'.format
plt.style.use('default')
np.set_printoptions(threshold = 30, edgeitems = 30, precision = 2, suppress = False)


In [3]:
df_path = "../merged_data/BCI_GDP.csv"
features = []
for i in range(14):
    features.extend([f'BCI_{i}', f'BCIp_{i}', f'BCIg_{i}'])
model_names = ["Linear Regression"]
get_models = [lambda: linear_model.LinearRegression()]


In [4]:
# Read the data and do a little bit of wrangling:
df = pd.read_csv(df_path)
df.Date = pd.to_datetime(df.Date)
df = df.set_index("Date", drop=True)
df = df.drop(columns="Unnamed: 0")
df.head()

,GDP,BCI_0,BCIp_0,BCIg_0,BCI_1,BCIp_1,BCIg_1,BCI_2,BCIp_2,BCIg_2,BCI_3,BCIp_3,BCIg_3,BCI_4,BCIp_4,BCIg_4,BCI_5,BCIp_5,BCIg_5,BCI_6,BCIp_6,BCIg_6,BCI_7,BCIp_7,BCIg_7,BCI_8,BCIp_8,BCIg_8,BCI_9,BCIp_9,BCIg_9,BCI_10,BCIp_10,BCIg_10,BCI_11,BCIp_11,BCIg_11,BCI_12,BCIp_12,BCIg_12,BCI_13,BCIp_13,BCIg_13
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1967-01-31,844.1700,100.0000,100.0000,0.0000,100.0000,99.5000,0.0000,99.6000,93.1000,0.0000,99.8000,96.4000,0.0000,99.9000,98.7000,0.0000,99.7000,94.6000,0.0000,99.9000,98.4000,0.0000,99.9000,97.6000,0.0000,99.2000,85.8000,0.0000,98.5000,75.1000,0.0000,98.9000,82.2000,0.0000,99.8000,96.6000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1967-04-30,848.9830,100.4000,100.0000,0.0000,100.5000,100.0000,0.0000,100.1000,93.4000,0.0000,99.8000,88.3000,0.0000,100.2000,95.6000,0.0000,100.6000,100.0000,0.0000,101.4000,100.0000,0.0000,101.8000,100.0000,0.0000,102.0000,100.0000,0.0000,102.3000,100.0000,0.0000,103.1000,100.0000,0.0000,103.2000,100.0000,0.0000,103.2000,99.8000,0.0000,0.0000,0.0000,0.0000
1967-07-31,865.2330,103.4000,100.0000,0.0000,103.4000,99.9000,0.0000,103.4000,100.0000,0.0000,103.6000,100.0000,0.0000,103.5000,98.0000,0.0000,104.0000,100.0000,0.0000,104.3000,100.0000,0.0000,104.9000,100.0000,0.0000,104.9000,100.0000,0.0000,104.7000,96.7000,0.0000,104.9000,99.7000,0.0000,104.5000,93.0000,0.0000,104.5000,93.1000,0.0000,0.0000,0.0000,0.0000
1967-10-31,881.4390,104.6000,94.3000,0.0000,104.2000,87.9000,0.0000,104.1000,87.5000,0.0000,104.7000,97.2000,0.0000,106.1000,100.0000,0.0000,106.7000,100.0000,0.0000,107.3000,100.0000,0.0000,107.4000,100.0000,0.0000,106.6000,87.4000,0.0000,106.2000,81.2000,0.0000,106.3000,82.3000,0.0000,106.4000,83.7000,0.0000,105.6000,71.7000,0.0000,0.0000,0.0000,0.0000
1968-01-31,909.3870,105.0000,62.7000,12.3000,104.9000,60.5000,11.3000,104.2000,49.2000,10.1000,104.2000,49.3000,9.2000,104.3000,51.9000,8.7000,103.6000,40.9000,7.9000,104.1000,48.7000,7.7000,104.1000,48.5000,7.5000,104.7000,57.4000,7.5000,105.9000,76.7000,8.5000,107.2000,96.9000,9.8000,108.5000,100.0000,11.7000,108.3000,97.3000,13.1000,13.1000,13.1000,13.1000


In [5]:
# Split into training and test sets and hold out the test set until the end, so that it remains "unseen".
lag_of_y = 2 # This is the lag we introduce to the target variable so that we assess the indicator's 
              # ability to predict the target variable this many steps into the future.
              # With GDP_m, a lag of 2 data points corresponds to half a year.
        
X_train, y_train = df.iloc[:-lag_of_y, df.columns != "GDP"], df.iloc[lag_of_y:, df.columns == "GDP"]

In [6]:
# for feature in features:
#     plt.figure()
#     X_train[feature].hist(bins = 50)
#     plt.xlabel(feature,fontsize=15)
#     plt.ylabel("Frequency",fontsize=15)
#     plt.show()

In [7]:
# Do a time series cross-validation on the test set by splitting it to k folds and doing a "rolling"
# validation against a validation fold, then averaging out the metrics.
splits = 4 # This is the number of splits/folds in the rolling validation.
tscv = TimeSeriesSplit(n_splits=splits)

for train_index, test_index in tscv.split(X_train): # Rolling cross-validation happens inside this loop.
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43] TEST: [44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67
 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 ... 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
 74 75 76 77 78 79 80 81 82 83 84 85 86] TEST: [ 87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104
 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122
 123 124 125 126 127 128 129]
TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29 ... 100 101 102 103 104
 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122
 123 124 125 126 127 128 129] TEST: [130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147
 148 149

## Validation

In [8]:
R2s = dict()
for model_name, get_model in zip(model_names, get_models):
    print(model_name)
    R2s[model_name] = 0
    for train_index, test_index in tscv.split(X_train): # Rolling cross-validation happens inside this loop.
        X_train_fold, X_validation_fold = X_train.iloc[train_index[:-lag_of_y], X_train.columns != "GDP"], \
            X_train.iloc[test_index[:-lag_of_y], X_train.columns != "GDP"]
        y_train_fold, y_validation_fold = y_train.iloc[train_index[lag_of_y:], y_train.columns == "GDP"], \
            y_train.iloc[test_index[lag_of_y:], y_train.columns == "GDP"]
            
        scalers = dict()
        for feature in features:
            scalers[feature] = StandardScaler()
            scalers[feature].fit(X_train_fold[[feature]])
            X_train_fold[feature] = scalers[feature].transform(X_train_fold[[feature]])
            X_validation_fold[feature] = scalers[feature].transform(X_validation_fold[[feature]])
            
        model = get_model()
        model.fit(X_train_fold[features], y_train_fold["GDP"])
        predictions = model.predict(X_validation_fold[features])
        R2 = r2_score(y_validation_fold, predictions)
        R2s[model_name] += R2
        print(R2)
        
    R2s[model_name] /= splits

Linear Regression
-26.06639163330296
-18.916153293791005
-5.0659637392593835
-76033.30885363856


In [9]:
for model_name in model_names:
    print(model_name)
    print(f"R2 score: {R2s[model_name]}")

Linear Regression
R2 score: -19020.83934057623
